In [2]:
def repeat(word, n):
    print(word * n)

repeat("spam, ", 4);

spam, spam, spam, spam, 


In [3]:
for line in open("res/word.txt"):
    print(line.strip())

Today is a good day.
Tomorrow will be fine.


In [4]:
word = 'Gadsby'
'e' in word

False

In [13]:
from PyPDF2 import PdfReader, PdfWriter, Transformation
reader = PdfReader("res/June.pdf")
page = reader.pages[3]
# print(page)
print(page.cropbox)
# RectangleObject([0, 0, 1233.72, 841.92])
lower_left = page.cropbox.lower_left
print(lower_left)
left_page = lower_left + (page.cropbox.upper_right[0]/2, page.cropbox.upper_right[1])
print(left_page)
right_page = (page.cropbox.upper_right[0]/2, page.cropbox.lower_left[1]) + page.cropbox.upper_right
print(right_page)
writer_left = PdfWriter()
writer_right = PdfWriter()
new_left_page = writer_left.add_blank_page(width=left_page[2]-left_page[0], height=left_page[3]-left_page[1])
new_left_page.add_transformation(Transformation().scale(1).translate(tx=-left_page[0], ty=-left_page[1]))
page.merge_page(new_left_page)

writer_left.add_page(new_left_page)

with open("res/" +"pdf_left1.pdf", "wb") as fp:
    writer_left.write(fp)

RectangleObject([0, 0, 841.2, 1227.12])
(0, 0)
(0, 0, Decimal('420.6'), 1227.12)
(Decimal('420.6'), 0, 841.2, 1227.12)


In [16]:
from PyPDF2 import PdfReader, PdfWriter, Transformation

reader = PdfReader("res/June.pdf")
page = reader.pages[3]

# 获取页面尺寸
cropbox = page.cropbox
width = float(cropbox.width)
height = float(cropbox.height)
print(f"Page size: {width} x {height}")

# 定义左半和右半的裁剪区域
left_bbox = [0, 0, width / 2, height]      # 左半
right_bbox = [width / 2, 0, width, height] # 右半

writer_left = PdfWriter()
writer_right = PdfWriter()

# 创建两个空白页（大小为原页面的一半）
# new_left_page = writer_left.add_blank_page(width=left_bbox[2]-left_bbox[0], height=height)
# new_right_page = writer_right.add_blank_page(width=right_bbox[2]-right_bbox[0], height=height)
#
# # 将原页面内容合并到左页，并向左平移，使右半部分被裁掉
# page_mutable = page.clone()
# # 向左移动半个页面宽度，这样右边的内容就移出可视区域
# trans_left = Transformation().translate(tx=-width/2, ty=0)
# page_mutable.add_transformation(trans_left)
# new_left_page.merge_page(page_mutable)
#
# # 将原页面内容合并到右页，并向左移动，使左半部分被裁掉
# page_mutable2 = page.clone()
# trans_right = Transformation().translate(tx=-width, ty=0)  # 移动整个宽度，再裁右边？
# 更准确的做法是先合并，再裁剪显示区域

# 更推荐方式：使用 scale + translate 实现裁剪效果
# 方法：缩放为1，然后平移，使得只显示目标区域

# # 正确方式：使用 Transformation 裁剪显示区域
def create_cropped_page(source_page, crop_rect, writer):
    """
    从 source_page 中提取 crop_rect 区域（[x1, y1, x2, y2]）
    """
    x1, y1, x2, y2 = crop_rect
    w = x2 - x1
    h = y2 - y1

    new_page = writer.add_blank_page(width=w, height=h)
    trans = Transformation().translate(tx=-x1, ty=-y1)
    new_page.merge_page(source_page)
    new_page.add_transformation(trans)
    return new_page

# 生成左右两页
new_left_page = create_cropped_page(page, left_bbox, writer_left)
new_right_page = create_cropped_page(page, right_bbox, writer_right)

writer_left.add_page(new_left_page)
writer_right.add_page(new_right_page)

# 保存
with open("res/pdf_left.pdf", "wb") as fp:
    writer_left.write(fp)

with open("res/pdf_right.pdf", "wb") as fp:
    writer_right.write(fp)

print("Split complete: pdf_left.pdf and pdf_right.pdf")

Page size: 841.2 x 1227.12
Split complete: pdf_left.pdf and pdf_right.pdf


# Another version of splitting

In [11]:
from pypdf import PdfReader, PdfWriter
reader = PdfReader("res/June.pdf")
rot = int(page.get("/Rotate", 0) or 0)

print(f"rotation: {rot}")

rotation: 0


In [15]:
# python
from pypdf import PdfReader, PdfWriter
import copy

reader = PdfReader("res/June.pdf")
writer = PdfWriter()

for index, page in enumerate(reader.pages):
    rot = int(page.get("/Rotate", 0) or 0)
    print(f"page: {index+1}, rotation: {rot}")
    # prefer CropBox if present, otherwise MediaBox
    box = page.cropbox if "/CropBox" in page else page.mediabox

    left = float(box.left)
    right = float(box.right)
    bottom = float(box.bottom)
    top = float(box.top)

    if rot % 180 == 0:
        # normal orientation: split by vertical center (x)
        mid = (left + right) / 2.0

        left_page = copy.deepcopy(page)
        # ensure both boxes reflect the new visible area
        left_page.mediabox.right = mid
        left_page.cropbox.right = mid
        writer.add_page(left_page)

        right_page = copy.deepcopy(page)
        right_page.mediabox.left = mid
        right_page.cropbox.left = mid
        writer.add_page(right_page)
    else:
        # rotated 90 or 270: split by horizontal center in PDF coords (y)
        mid = (bottom + top) / 2.0

        left_page = copy.deepcopy(page)
        left_page.mediabox.top = mid
        left_page.cropbox.top = mid
        writer.add_page(left_page)

        right_page = copy.deepcopy(page)
        right_page.mediabox.bottom = mid
        right_page.cropbox.bottom = mid
        writer.add_page(right_page)

with open("res/June_split.pdf", "wb") as out_f:
    writer.write(out_f)

page: 1, rotation: 0
page: 2, rotation: 90
page: 3, rotation: 270
page: 4, rotation: 90
page: 5, rotation: 270
page: 6, rotation: 90
page: 7, rotation: 270
page: 8, rotation: 90
page: 9, rotation: 270
page: 10, rotation: 90
page: 11, rotation: 270
page: 12, rotation: 90
page: 13, rotation: 270
page: 14, rotation: 90
page: 15, rotation: 270
page: 16, rotation: 90
page: 17, rotation: 270
page: 18, rotation: 90
page: 19, rotation: 270
page: 20, rotation: 90
page: 21, rotation: 270
page: 22, rotation: 90
page: 23, rotation: 270
page: 24, rotation: 90
page: 25, rotation: 270
page: 26, rotation: 90
page: 27, rotation: 270
page: 28, rotation: 90
page: 29, rotation: 270
page: 30, rotation: 90
page: 31, rotation: 270
page: 32, rotation: 90
page: 33, rotation: 270
page: 34, rotation: 90
page: 35, rotation: 270
page: 36, rotation: 90
page: 37, rotation: 270
page: 38, rotation: 90
page: 39, rotation: 270
page: 40, rotation: 90
page: 41, rotation: 270
page: 42, rotation: 90
page: 43, rotation: 270


In [13]:
# python
from pypdf import PdfReader, PdfWriter
import copy

reader = PdfReader("res/June.pdf")
writer = PdfWriter()

for page in reader.pages:
    # get numeric coordinates
    left_x = float(page.mediabox.left)
    right_x = float(page.mediabox.right)
    mid_x = (left_x + right_x) / 2.0

    # left half
    left_page = copy.deepcopy(page)
    left_page.mediabox.right = mid_x
    left_page.cropbox.right = mid_x
    writer.add_page(left_page)

    # right half
    right_page = copy.deepcopy(page)
    right_page.mediabox.left = mid_x
    right_page.cropbox.left = mid_x
    writer.add_page(right_page)

with open("res/June_split_test.pdf", "wb") as out_f:
    writer.write(out_f)